%md
source API URL : "https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2023-01-01&end_date=2024-01-01&daily=temperature_2m_max,temperature_2m_min,rain_sum"

JSON Target File Path : "abfss://bronze@datalakestorageaccountname.dfs.core.windows.net/bronze/weather-data/
"


In [0]:
weatherDataSinkLayerName = "bronze"
weatherDataStorageAccountName = "adlsudadalakehousedev"
weatherDataSinkFolderName = "weather-data"

weatherDataSinkFolderPath = f"abfss://{weatherDataSinkLayerName}@{weatherDataStorageAccountName}.dfs.core.windows.net/{weatherDataSinkFolderName}"

In [0]:
import requests
import json 
import pandas as pd

In [0]:
geoLocationDF = spark.sql("select latitude, longitude, marketName from pricing_analytics.silver.geo_location_silver limit 100 ")

In [0]:
geoLocationDF.count()

In [0]:

weatherDataAPIResponseList = []
for latitude, longitude, marketname in geoLocationDF.collect():
    base_url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "marketname": marketname,
        "start_date": "2023-01-01",
        "end_date": "2023-12-31",
        "daily": "temperature_2m_max,temperature_2m_min,rain_sum",
    }

    response = requests.get(base_url, params=params, timeout= 120).json()
    response["marketName"] = marketname
    weatherDataAPIResponseList.append(response)


   


In [0]:
weatherDataPandasDF= pd.DataFrame(weatherDataAPIResponseList)
weatherDataAPISpark = spark.createDataFrame(weatherDataPandasDF)

In [0]:
display(weatherDataAPISpark)

In [0]:
(weatherDataAPISpark
 .write
 .mode("overwrite")
 .json(weatherDataSinkFolderPath))

In [0]:
dbutils.fs.ls(weatherDataSinkFolderPath)